<h1>
Testing accuracy of Stock Price Predictions made using XGBOOST<h1>

In [1]:
!pip install scikit-learn
!pip install pandas
!pip install plotly
import sys
!{sys.executable} -m pip install xgboost

'C:\Users\Nirzar' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
from IPython.core.debugger import set_trace

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
import time
import yfinance as yf
plt.style.use(style="seaborn")
%matplotlib inline

In [3]:
df = pd.read_csv("C:/Users/Nirzar Panvelkar/OneDrive/dm proj/TSLA-max-hourly.csv")
df.dtypes

Date          object
Close/Last    object
Volume         int64
Open          object
High          object
Low           object
dtype: object

<h3>Data cleaning and Preprocessing</h3>

In [4]:
#changing data types
df['Date'] = pd.to_datetime(df['Date'])
df['Close/Last'] = df['Close/Last'].replace({'\$':''}, regex = True)
df['Close/Last'] = pd.to_numeric(df['Close/Last'])
df['Open'] = df['Open'].replace({'\$':''}, regex = True)
df['Open'] = pd.to_numeric(df['Open'])
df['High'] = df['High'].replace({'\$':''}, regex = True)
df['High'] = pd.to_numeric(df['High'])
df['Low'] = df['Low'].replace({'\$':''}, regex = True)
df['Low'] = pd.to_numeric(df['Low'])
df.dtypes

Date          datetime64[ns]
Close/Last           float64
Volume                 int64
Open                 float64
High                 float64
Low                  float64
dtype: object

In [5]:
df.head(5)

,Date,Close/Last,Volume,Open,High,Low
0,2021-07-15,650.60,20209570,658.390,666.1400,637.8800
1,2021-07-14,653.38,21641190,670.750,678.6099,652.8400
2,2021-07-13,668.54,20966090,686.320,693.2800,666.2958
3,2021-07-12,685.70,25927040,662.200,687.2400,662.1610
4,2021-07-09,656.95,18140550,653.185,658.9100,644.6900


In [6]:
df_ = df[['Close/Last']].copy()
df_.shape

(2516, 1)

In [7]:
df.head(5)

,Date,Close/Last,Volume,Open,High,Low
0,2021-07-15,650.60,20209570,658.390,666.1400,637.8800
1,2021-07-14,653.38,21641190,670.750,678.6099,652.8400
2,2021-07-13,668.54,20966090,686.320,693.2800,666.2958
3,2021-07-12,685.70,25927040,662.200,687.2400,662.1610
4,2021-07-09,656.95,18140550,653.185,658.9100,644.6900


In [8]:
df_["target"] = df_['Close/Last'].shift(-1)

In [9]:
df_.dropna(inplace=True) #drop nan values
df_.shape

(2515, 2)

In [10]:
df_.head(5)

,Close/Last,target
0,650.60,653.38
1,653.38,668.54
2,668.54,685.70
3,685.70,656.95
4,656.95,652.81


<h3>Train Test data Split</h3>

In [11]:
def train_test_split(data, perc):
    data = data.values
    n = int(len(data) * (1 - perc))
    return data[:n], data[n:]

In [12]:
train, test = train_test_split(df_, 0.2)  #80 percent training, 20 percent testing
print(len(train))
print(len(test))

2012
503


In [13]:
X = train[:, :-1]
y = train[:, -1]
X

array([[650.6  ],
       [653.38 ],
       [668.54 ],
       ...,
       [ 24.548],
       [ 24.486],
       [ 23.936]])

In [14]:
y

array([653.38 , 668.54 , 685.7  , ...,  24.486,  23.936,  23.806])

In [15]:
model = XGBRegressor(objective="reg:squarederror", n_estimators=1000)
model.fit(X, y)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=1000,
             n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
             reg_alpha=0, reg_lambda=1, ...)

In [16]:
test[0]

array([23.806, 24.05 ])

In [17]:
val = np.array(test[0, 0]).reshape(1, -1)

pred = model.predict(val)
print(pred[0])

23.835068


<h3>Predict one sample at a time</h3>

In [18]:
def xgb_predict(train, val):
    train = np.array(train)
    X, y = train[:, :-1], train[:, -1]
    model = XGBRegressor(objective="reg:squarederror", n_estimators=1000)
    model.fit(X, y)

    val = np.array(val).reshape(1, -1)
    pred = model.predict(val)
    return pred[0]

In [19]:
xgb_predict(train, test[0, 0])

23.835068

In [20]:
from sklearn.metrics import mean_squared_error


def validate(data, perc):
    predictions = []

    train, test = train_test_split(data, perc)

    history = [x for x in train]

    for i in range(len(test)):
        test_X, test_y = test[i, :-1], test[i, -1]

        pred = xgb_predict(history, test_X[0])
        predictions.append(pred)

        history.append(test[i])

    error = mean_squared_error(test[:, -1], predictions, squared=False)

    return error, test[:, -1], predictions

<h4>So our predicted model has low RMSE. But still it is not the most accurate model to predict stock price trend. But for long term investements, one may trust this predictions.</h4>

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
print("Accuracy on Train Data: ", round(accuracy_score(train, yp_train),4))
